In [2]:
# -*- coding: utf-8 -*-

# import faiss
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoModel, AutoProcessor, CLIPModel
from PIL import Image
import os
from tqdm import tqdm

In [5]:
images_dir_path = r'E:\work\spatio_evo_urbanvisenv_svi\patch_kmeans\sv_split\sv_400_400'
images_dir_path = r'E:\work\spatio_evo_urbanvisenv_svi\patch_kmeans\sv_split\sv_grey'
# 图片库所在文件夹位置
roots = []
img_names = []
img_paths = []

accepted_formats = (".png", ".jpg", ".JPG", ".jpeg")
for root, dirs, files in os.walk(images_dir_path):
    for file in files:
        if file.endswith(accepted_formats):
            roots.append(root)
            img_names.append(file)
            file_path = os.path.join(root, file)
            img_paths.append(file_path)
print(len(img_names))
print(len(roots))

KeyboardInterrupt: 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

#Load CLIP model and processor
# processor_clip = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
# model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

#Load DINOv2 model and processor
processor_dino = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model_dino = AutoModel.from_pretrained('facebook/dinov2-base').to(device)


In [3]:
#Define a function that normalizes embeddings and add them to the index
def add_vector_to_index(embedding, index):
    #convert embedding to numpy
    vector = embedding.detach().cpu().numpy()
    #Convert to float32 numpy
    vector = np.float32(vector)
    #Normalize vector: important to avoid wrong results when searching
    faiss.normalize_L2(vector)
    #Add to index
    index.add(vector)

# def extract_features_clip(image):
#     with torch.no_grad():
#         inputs = processor_clip(images=image, return_tensors="pt").to(device)
#         image_features = model_clip.get_image_features(**inputs)
#         return image_features

def extract_features_dino(image):
    with torch.no_grad():
        inputs = processor_dino(images=image, return_tensors="pt").to(device)
        outputs = model_dino(**inputs)
        image_features = outputs.last_hidden_state
        return image_features.mean(dim=1)

#Create 2 indexes.
# index_clip = faiss.IndexFlatL2(512)
# index_dino = faiss.IndexFlatL2(768)

In [4]:
classes_lcz = ['480','320']
for class_lcz in classes_lcz:
    index_dino = faiss.IndexFlatL2(768)

    # 图片库所在文件夹位置
    roots = []
    img_names = []
    img_paths = []

    accepted_formats = (".png", ".jpg", ".JPG", ".jpeg")

    images_dir_path=r"F:\lcz\\" + class_lcz
    for root, dirs, files in os.walk(images_dir_path):
        for file in files:
            if file.endswith(accepted_formats):
                roots.append(root)
                img_names.append(file)
                file_path = os.path.join(root, file)
                img_paths.append(file_path)
    print(len(img_names))
    print(len(roots))
    #Iterate over the dataset to extract features X2 and store features in indexes
    for image_path in tqdm(img_paths):
        img = Image.open(image_path).convert('RGB')
        # clip_features = extract_features_clip(img)
        # add_vector_to_index(clip_features,index_clip)
        dino_features = extract_features_dino(img)
        add_vector_to_index(dino_features,index_dino)

    #store the indexes locally
    # faiss.write_index(index_clip,"clip.index")
    faiss.write_index(index_dino,f"F:\lcz\\dino_{class_lcz}.index")


173900
173900


100%|██████████| 173900/173900 [2:48:05<00:00, 17.24it/s]  


390165
390165


100%|██████████| 390165/390165 [5:57:33<00:00, 18.19it/s]  


In [6]:
index_dino = faiss.read_index(f"F:\lcz\\dino_320.index")

# faiss.write_index(index_dino,f"F:\lcz\\dino_{class_lcz}.index")
index_dino.ntotal



390165

In [43]:
import os
import shutil

# 目标文件夹
patch_ratio = '320'
roots = []
img_names = []
img_paths = []

accepted_formats = (".png", ".jpg", ".JPG", ".jpeg")

images_dir_path=f"F:\\lcz\\{patch_ratio}"
for root, dirs, files in os.walk(images_dir_path):
    for file in files:
        if file.endswith(accepted_formats):
            roots.append(root)
            img_names.append(file)
            file_path = os.path.join(root, file)
            img_paths.append(file_path)
#Input image
for i in range(18):
    if i==0:
        continue
    destination_folder = f'F:\\lcz\\lcz_train\\{patch_ratio}\lcz{i}'
    # 确保目标文件夹存在
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    source=f'f:\\lcz\\lcz_train\\nan\\lcz{i}.png'
    image = Image.open(source)

    #Extract features for CLIP
    # with torch.no_grad():
    #     inputs_clip = processor_clip(images=image, return_tensors="pt").to(device)
    #     image_features_clip = model_clip.get_image_features(**inputs_clip)

    #Extract features for DINOv2
    with torch.no_grad():
        inputs_dino = processor_dino(images=image, return_tensors="pt").to(device)
        outputs_dino = model_dino(**inputs_dino)
        image_features_dino = outputs_dino.last_hidden_state
        image_features_dino = image_features_dino.mean(dim=1)

    def normalizeL2(embeddings):
        vector = embeddings.detach().cpu().numpy()
        vector = np.float32(vector)
        faiss.normalize_L2(vector)
        return vector

    image_features_dino = normalizeL2(image_features_dino)
    # image_features_clip = normalizeL2(image_features_clip)
    #Search the top 5 images
    # index_clip = faiss.read_index("clip.index")
    index_dino = faiss.read_index(f"F:\lcz\\dino_{patch_ratio}.index")

    #Get distance and indexes of images associated
    d_dino,i_dino = index_dino.search(image_features_dino,500)
    # d_clip,i_clip = index_clip.search(image_features_clip,5)
    for i in tqdm(i_dino[0]):
        file_name = os.path.basename(img_paths[i])
        # 目标文件路径
        destination_path = os.path.join(destination_folder, file_name)
        # 复制文件
        shutil.copy(img_paths[i], destination_path)
        # print(f'文件 {file_name} 已复制到 {destination_folder}')


100%|██████████| 500/500 [00:01<00:00, 325.79it/s]


In [25]:
# 图片库所在文件夹位置




F:\lcz\320\4\cropped_16353_88389.png
F:\lcz\320\129\cropped_10764_91494.png
F:\lcz\320\105\cropped_16353_87975.png
F:\lcz\320\105\cropped_10971_88596.png
F:\lcz\320\105\cropped_11178_88596.png


In [42]:
index_dino.ntotal

662199

In [37]:
index_dino.reconstruct(0)

array([-2.34308466e-02, -3.94508243e-02,  2.82763541e-02, -6.73424453e-02,
       -3.22279781e-02,  2.46672090e-02,  9.98163316e-03, -1.68999732e-02,
        1.96109787e-02,  2.40171365e-02,  1.86087340e-02,  3.09300832e-02,
       -4.23079096e-02, -2.63395230e-03,  5.55535220e-02, -3.14871706e-02,
       -4.12225388e-02,  4.50028069e-02,  3.12393866e-02,  1.21453882e-03,
        4.25007474e-03, -2.58767344e-02,  2.71617752e-02,  3.82540151e-02,
        3.96389253e-02, -7.85629153e-02,  4.18864451e-02, -5.44557720e-03,
        2.92054527e-02,  4.19009402e-02,  1.19411135e-02,  3.57641280e-02,
        1.66312344e-02, -1.48065831e-03, -2.74692383e-02, -5.00465147e-02,
       -2.58328766e-02,  3.26316468e-02,  5.73345351e-05,  2.76057608e-02,
        3.36659327e-02,  4.52460125e-02,  5.83592653e-02, -6.11172244e-02,
       -1.37706958e-02,  4.00452986e-02, -9.96730477e-03,  2.40340978e-02,
       -7.26190805e-02, -6.46187738e-03,  6.34755567e-02, -6.98486809e-04,
       -1.25472788e-02,  

In [29]:
# 使用 i_dino 中的索引来检索特征向量
for j, idx in enumerate(i_dino[0]):
    # print(j)
    print(idx)

272036
272041
272056
272323
275512
